In [ ]:
# [SET KERAS BACKEND]
import os 
os.environ["KERAS_BACKEND"] = "torch"

# [IMPORT LIBRARIES]
import pandas as pd

# set warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

# import modules and classes
from XREPORT.commons.utils.process.tokenizers import TokenWizard
from XREPORT.commons.utils.process.splitting import TrainValidationSplit
from XREPORT.commons.utils.dataloader.tensordata import TensorDatasetBuilder
from XREPORT.commons.utils.dataloader.serializer import DataSerializer, ModelSerializer
from XREPORT.commons.utils.validation.reports import evaluation_report
from XREPORT.commons.constants import CONFIG, DATA_PATH
from XREPORT.commons.logger import logger

# Load data and model

In [ ]:
# selected and load the pretrained model, then print the summary 
modelserializer = ModelSerializer()         
model, configuration, history, checkpoint_path = modelserializer.select_and_load_checkpoint()
model.summary(expand_nested=True)   

# load data from csv, add paths to images 
dataserializer = DataSerializer(configuration)
processed_data, metadata = dataserializer.load_data_from_checkpoint(checkpoint_path)

processed_data = dataserializer.get_images_path_from_dataset(processed_data)
vocabulary_size = metadata['vocabulary_size']

# initialize the TensorDataSet class with the generator instances
# create the tf.datasets using the previously initialized generators
splitter = TrainValidationSplit(configuration, processed_data)     
train_data, validation_data = splitter.split_train_and_validation()      

## Create generator and datasets

In [ ]:
# get tokenizers and its info
tokenization = TokenWizard(configuration)    
tokenizer = tokenization.tokenizer

# initialize the TensorDataSet class with the generator instances
# create the tf.datasets using the previously initialized generators    
builder = TensorDatasetBuilder(configuration)  
train_dataset, validation_dataset = builder.build_model_dataloader(train_data, validation_data, configuration)

# Model evaluation

### Evaluation of loss and metrics

In [ ]:
evaluation_report(model, train_dataset, validation_dataset)  